In [16]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

In [17]:
class CNNModel1(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 10, kernel_size = 5, stride = 1)
    self.conv2 = nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = 5, stride = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.lin1 = nn.Linear(in_features= 4*4*10,out_features = 100)
    self.lin2 = nn.Linear(100,10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = torch.flatten(x,1)
    x = F.relu(self.lin1(x))
    x = self.lin2(x)
    return x

In [ ]:
def get_accuracy(predictions, targets):
  length = len(predictions)
  correct = 0
  for idx in range(length):
    if predictions[idx] == targets[idx]: correct +=1

  return (correct/length) * 100

In [ ]:
train_x = train_x.unsqueeze(1)
validation_x = validation_x.unsqueeze(1)
test_x = test_x.unsqueeze(1) # unsqueezing to introduce batchsize

In [ ]:
model1 = CNNModel1()

In [ ]:
batch_size = 128
num_epochs = 20
num_batches_per_train_epoch = train_x.shape[0] // batch_size
num_batches_validation = validation_x.shape[0] // batch_size

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [20]:
def train_epoch(model, train_x, train_y, validation_x, validation_y):

  epoch_loss = 0

  model.train()
  train_preds = list()
  train_targets = list()


    # Train one epoch
  for batch_idx in range(num_batches_per_train_epoch):
    optimizer.zero_grad()  # This line is necessary to flush out the gradients of the previous batch.

    input = train_x[batch_idx*batch_size: (batch_idx+1)*batch_size] # Slice out batch_size amount of the training data
    output = model(input)
    target_out = train_y[batch_idx*batch_size: (batch_idx+1)*batch_size]
    preds = torch.argmax(output, dim=1)

    train_preds +=(list(preds.detach().cpu().numpy()))
    train_targets+=(list(target_out.detach().cpu().numpy()))
    batch_loss = criterion(output, target_out)

    batch_loss.backward()
    optimizer.step()
    epoch_loss += batch_loss
  epoch_loss_ret = epoch_loss.detach().cpu() / batch_size

    # Switch model to eval mode since we do not want to update our weights using test/val set images! They are for measuring performance only
  model.eval()
    # Training Performance at the end of epoch

  val_preds = list()
  val_targets = list()

  for batch_idx in range(num_batches_validation):
    input = validation_x[batch_idx*batch_size: (batch_idx+1)*batch_size]
    output = model(input)
    target_out = validation_y[batch_idx*batch_size: (batch_idx+1)*batch_size]
    # preds = torch.argmax(output, dim=1)
    preds = torch.max(output, 1)[1]


    val_preds += (list(preds.detach().cpu().numpy()))
    val_targets+=(list(target_out.detach().cpu().numpy()))

  train_accuracy = get_accuracy(train_preds, train_targets)
  val_accuracy = get_accuracy(val_preds, val_targets)

  return train_accuracy, val_accuracy, epoch_loss_ret

def train_model(train_x, train_y, validation_x, validation_y, model, num_epochs):
    # Train the model
  epoch_loss = 0
  losses_at_each_epoch = list()
  train_accuracies = list()
  validation_accuracies = list()

  # Forward pass -> Backward pass -> Weight update

  for epoch in range(num_epochs):
    train_accuracy, val_accuracy, epoch_loss = train_epoch(model, train_x, train_y, validation_x, validation_y)

    train_accuracies.append(train_accuracy)
    validation_accuracies.append(val_accuracy)
    losses_at_each_epoch.append(epoch_loss)

    print("Epoch %2i : Train Loss %f , Train acc %f, Valid acc %f" % (
                epoch, losses_at_each_epoch[-1], train_accuracies[-1], validation_accuracies[-1]))

  return model, epoch_loss, losses_at_each_epoch, train_accuracies, validation_accuracies